# StockData 数据查看器

本notebook用于展示StockData类中所有fetcher获取的实际数据内容，帮助快速了解每个股票相关接口的功能和数据结构。

## 📊 StockData包含的数据类型

### 基础股票数据
- **历史行情数据** - 股票的K线数据（开高低收量）
- **实时行情数据** - 股票当前的实时价格和交易信息
- **股票基本信息** - 股票名称、行业、市值等基本信息
- **筹码分布数据** - 股票的持仓成本分布

### 基本面数据
- **财务报表数据** - 利润表、资产负债表、现金流量表
- **分红数据** - 历史分红派息记录
- **业绩数据** - 营收、净利润等业绩指标
- **股东数据** - 十大股东持股情况
- **股本变动** - 增发、回购等股本变化
- **限售解禁** - 限售股解禁时间表
- **大宗交易** - 大宗交易记录

## 1. 导入库并初始化

In [1]:
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 获取项目根目录
current_dir = os.path.abspath('.')
project_root = os.path.dirname(current_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 导入StockData
from china_stock_data import StockData

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 10)

print("✅ 导入成功")
print(f"项目根目录: {project_root}")

✅ 导入成功
项目根目录: /Users/juzhongsun/Codes/projects/china_stock_data


## 2. 创建StockData实例

In [2]:
# 创建StockData实例 - 以贵州茅台(600519)为例
stock = StockData('600519')

print(f"StockData实例创建成功！")
print(f"股票代码: {stock.symbol}")
print(f"实例Key: {stock.key()}")
print(f"可用fetchers: {len(stock.fetchers)}个")

# 获取所有fetcher信息
unique_fetchers = {}
for name, fetcher in stock.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

print(f"唯一fetchers: {len(unique_fetchers)}个")
print(f"访问名称总数: {len(stock.fetchers)}个")

StockData实例创建成功！
股票代码: 600519
实例Key: 600519
可用fetchers: 27个
唯一fetchers: 12个
访问名称总数: 27个


## 3. 查看所有可用的Fetchers

In [3]:
print("📋 所有可用的Stock Data Fetchers:")
print("=" * 60)

# 按类别分组展示
basic_fetchers = []
fundamental_fetchers = []

for fetcher_id, info in unique_fetchers.items():
    class_name = info['class_name']
    if any(keyword in class_name.lower() for keyword in ['hist', 'info', 'chip', 'realtime']):
        basic_fetchers.append(info)
    else:
        fundamental_fetchers.append(info)

print("🔵 基础股票数据:")
for i, info in enumerate(basic_fetchers, 1):
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    print(f"  {i}. {class_name}")
    print(f"     访问名称: {all_names}")

print(f"\n🟢 基本面数据:")
for i, info in enumerate(fundamental_fetchers, 1):
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    print(f"  {i}. {class_name}")
    print(f"     访问名称: {all_names}")

print(f"\n📊 访问名称详细列表:")
fetcher_names = sorted(stock.fetchers.keys())
for i, name in enumerate(fetcher_names, 1):
    print(f"{i:2d}. {name}")

📋 所有可用的Stock Data Fetchers:
🔵 基础股票数据:
  1. StockHistFetcher
     访问名称: kline | stock_zh_a_hist
  2. StockInfoFetcher
     访问名称: info | stock_individual_info_em
  3. StockChipFetcher
     访问名称: chip | stock_cyq_em
  4. StockRealtimeFetcher
     访问名称: bid_ask | stock_bid_ask_em

🟢 基本面数据:
  1. BlockTradeFetcher
     访问名称: block_trade | stock_block_trade_big_em
  2. DividendFetcher
     访问名称: dividend | stock_fhps_em
  3. EarningsFetcher
     访问名称: earnings | stock_yjbb_em | stock_yjyg_em
  4. FinancialStatementsFetcher
     访问名称: financial_statements | stock_lrb_em | stock_zcfz_em | stock_xjll_em
  5. RepurchaseFetcher
     访问名称: repurchase | stock_repurchase_em
  6. RestrictedReleaseFetcher
     访问名称: restricted_release | stock_restricted_release_queue_em
  7. ShareChangeFetcher
     访问名称: share_change | stock_share_change_em
  8. TopShareholdersFetcher
     访问名称: top_shareholders | stock_gdfx_free_top_10_em

📊 访问名称详细列表:
 1. bid_ask
 2. block_trade
 3. chip
 4. dividend
 5. earnings
 6. 

## 4. 数据展示函数

In [4]:
def display_stock_fetcher_data(fetcher_name, description, stock_instance):
    """
    展示指定fetcher的数据内容和结构
    """
    print(f"\n{'='*80}")
    print(f"📊 {description}")
    print(f"🔗 访问名称: {fetcher_name}")
    print(f"📍 股票代码: {stock_instance.symbol}")
    print(f"{'='*80}")
    
    try:
        # 获取数据
        data = stock_instance.get_data(fetcher_name)
        
        if data.empty:
            print("❌ 数据为空")
            return
        
        # 基本信息
        print(f"📈 数据形状: {data.shape[0]}行 x {data.shape[1]}列")
        print(f"📋 列名: {list(data.columns)}")
        
        # 数据类型
        print(f"\n🔢 数据类型:")
        for col, dtype in data.dtypes.items():
            print(f"   {col}: {dtype}")
        
        # 数据预览
        print(f"\n📄 数据预览 (前5行):")
        print(data.head())
        
        if len(data) > 10:
            print(f"\n📄 数据预览 (后5行):")
            print(data.tail())
        
        # 数值列的统计信息
        numeric_cols = data.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0 and len(numeric_cols) <= 10:  # 避免过多列的统计
            print(f"\n📊 数值列统计:")
            print(data[numeric_cols].describe())
        elif len(numeric_cols) > 10:
            print(f"\n📊 数值列统计 (仅显示前5列):")
            print(data[numeric_cols.tolist()[:5]].describe())
        
        # 特殊信息提取
        if 'date' in [col.lower() for col in data.columns] or '日期' in str(data.columns):
            date_cols = [col for col in data.columns if 'date' in col.lower() or '日期' in col]
            if date_cols:
                date_col = date_cols[0]
                if not data[date_col].empty:
                    try:
                        print(f"\n📅 时间范围: {data[date_col].min()} 至 {data[date_col].max()}")
                    except:
                        pass
        
        # K线数据特殊处理
        price_cols = [col for col in data.columns if any(keyword in col.lower() for keyword in ['price', 'close', '收盘', '价格'])]
        if price_cols and len(data) > 1:
            price_col = price_cols[0]
            try:
                latest_price = data[price_col].iloc[-1]
                price_change = data[price_col].iloc[-1] - data[price_col].iloc[-2]
                change_pct = (price_change / data[price_col].iloc[-2]) * 100
                print(f"\n💰 最新价格: {latest_price:.2f}")
                print(f"📈 价格变化: {price_change:+.2f} ({change_pct:+.2f}%)")
            except:
                pass
        
        print(f"\n✅ 数据获取成功")
        
    except Exception as e:
        print(f"❌ 获取数据时出错: {str(e)}")

## 5. 基础股票数据

### 5.1 历史行情数据

In [5]:
display_stock_fetcher_data('kline', '历史行情数据 - 股票的K线数据，包含开高低收量等信息', stock)


📊 历史行情数据 - 股票的K线数据，包含开高低收量等信息
🔗 访问名称: kline
📍 股票代码: 600519
📈 数据形状: 242行 x 14列
📋 列名: ['日期', '股票代码', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率', '平均', '加权平均']

🔢 数据类型:
   日期: object
   股票代码: int64
   开盘: float64
   收盘: float64
   最高: float64
   最低: float64
   成交量: int64
   成交额: float64
   振幅: float64
   涨跌幅: float64
   涨跌额: float64
   换手率: float64
   平均: float64
   加权平均: float64

📄 数据预览 (前5行):
           日期    股票代码       开盘       收盘       最高       最低     成交量  \
0  2024-10-08  600519  1858.52  1671.52  1858.52  1628.70  194709   
1  2024-10-09  600519  1648.52  1543.67  1648.52  1538.52  120197   
2  2024-10-10  600519  1570.78  1588.52  1616.60  1539.52   74603   
3  2024-10-11  600519  1563.92  1553.51  1588.48  1538.52   41915   
4  2024-10-14  600519  1561.52  1550.51  1569.15  1529.69   44812   

            成交额     振幅   涨跌幅     涨跌额   换手率       平均         加权平均  
0  3.511408e+10  13.55 -1.47  -25.00  1.55  1743.61  1803.413042  
1  1.970843e+10   6.58 -7.65 -127.

### 5.2 实时行情数据

In [6]:
display_stock_fetcher_data('realtime', '实时行情数据 - 股票当前的实时价格和交易信息', stock)


📊 实时行情数据 - 股票当前的实时价格和交易信息
🔗 访问名称: realtime
📍 股票代码: 600519
❌ 获取数据时出错: Unknown data type: realtime


### 5.3 股票基本信息

In [7]:
display_stock_fetcher_data('info', '股票基本信息 - 股票名称、行业、市值等基本信息', stock)


📊 股票基本信息 - 股票名称、行业、市值等基本信息
🔗 访问名称: info
📍 股票代码: 600519
📈 数据形状: 8行 x 2列
📋 列名: ['item', 'value']

🔢 数据类型:
   item: object
   value: object

📄 数据预览 (前5行):
   item            value
0  股票代码           600519
1  股票简称             贵州茅台
2   总股本     1252270215.0
3   流通股     1252270215.0
4   总市值  1797007758525.0

✅ 数据获取成功


### 5.4 筹码分布数据

In [8]:
display_stock_fetcher_data('chip', '筹码分布数据 - 股票的持仓成本分布情况', stock)


📊 筹码分布数据 - 股票的持仓成本分布情况
🔗 访问名称: chip
📍 股票代码: 600519
📈 数据形状: 90行 x 9列
📋 列名: ['日期', '获利比例', '平均成本', '90成本-低', '90成本-高', '90集中度', '70成本-低', '70成本-高', '70集中度']

🔢 数据类型:
   日期: object
   获利比例: float64
   平均成本: float64
   90成本-低: float64
   90成本-高: float64
   90集中度: float64
   70成本-低: float64
   70成本-高: float64
   70集中度: float64

📄 数据预览 (前5行):
           日期      获利比例     平均成本   90成本-低   90成本-高     90集中度   70成本-低  \
0  2025-05-23  0.808443  1493.61  1407.04  1595.76  0.062849  1433.01   
1  2025-05-26  0.677865  1495.34  1407.04  1595.76  0.062849  1433.01   
2  2025-05-27  0.647551  1495.34  1407.04  1595.76  0.062849  1433.01   
3  2025-05-28  0.606472  1495.34  1407.04  1595.76  0.062849  1433.01   
4  2025-05-29  0.616164  1497.07  1407.04  1595.76  0.062849  1433.01   

    70成本-高     70集中度  
0  1554.21  0.040572  
1  1554.21  0.040572  
2  1554.21  0.040572  
3  1552.48  0.040016  
4  1552.48  0.040016  

📄 数据预览 (后5行):
            日期      获利比例     平均成本   90成本-低   90成本-高     90集中度   70成本

## 6. 基本面数据

### 6.1 财务报表数据

In [9]:
display_stock_fetcher_data('financial_statements', '财务报表数据 - 利润表、资产负债表、现金流量表', stock)


📊 财务报表数据 - 利润表、资产负债表、现金流量表
🔗 访问名称: financial_statements
📍 股票代码: 600519
❌ 数据为空


### 6.2 分红数据

In [10]:
display_stock_fetcher_data('dividend', '分红数据 - 历史分红派息记录', stock)


📊 分红数据 - 历史分红派息记录
🔗 访问名称: dividend
📍 股票代码: 600519
❌ 数据为空


### 6.3 业绩数据

In [11]:
display_stock_fetcher_data('earnings', '业绩数据 - 营收、净利润等业绩指标', stock)


📊 业绩数据 - 营收、净利润等业绩指标
🔗 访问名称: earnings
📍 股票代码: 600519
❌ 数据为空


### 6.4 股东数据

In [12]:
display_stock_fetcher_data('top_shareholders', '股东数据 - 十大股东持股情况', stock)


📊 股东数据 - 十大股东持股情况
🔗 访问名称: top_shareholders
📍 股票代码: 600519
❌ 数据为空


### 6.5 股本变动数据

In [13]:
display_stock_fetcher_data('share_change', '股本变动数据 - 增发、回购等股本变化记录', stock)


📊 股本变动数据 - 增发、回购等股本变化记录
🔗 访问名称: share_change
📍 股票代码: 600519
❌ 数据为空


### 6.6 回购数据

In [14]:
display_stock_fetcher_data('repurchase', '回购数据 - 股票回购计划和执行情况', stock)


📊 回购数据 - 股票回购计划和执行情况
🔗 访问名称: repurchase
📍 股票代码: 600519
📈 数据形状: 4868行 x 18列
📋 列名: ['序号', '股票代码', '股票简称', '最新价', '计划回购价格区间', '计划回购数量区间-下限', '计划回购数量区间-上限', '占公告前一日总股本比例-下限', '占公告前一日总股本比例-上限', '计划回购金额区间-下限', '计划回购金额区间-上限', '回购起始时间', '实施进度', '已回购股份价格区间-下限', '已回购股份价格区间-上限', '已回购股份数量', '已回购金额', '最新公告日期']

🔢 数据类型:
   序号: int64
   股票代码: int64
   股票简称: object
   最新价: float64
   计划回购价格区间: float64
   计划回购数量区间-下限: float64
   计划回购数量区间-上限: float64
   占公告前一日总股本比例-下限: float64
   占公告前一日总股本比例-上限: float64
   计划回购金额区间-下限: float64
   计划回购金额区间-上限: float64
   回购起始时间: object
   实施进度: object
   已回购股份价格区间-下限: float64
   已回购股份价格区间-上限: float64
   已回购股份数量: float64
   已回购金额: float64
   最新公告日期: object

📄 数据预览 (前5行):
   序号    股票代码  股票简称    最新价  计划回购价格区间  计划回购数量区间-下限  计划回购数量区间-上限  \
0   1  688166  博瑞医药  58.06    139.77      71500.0     143100.0   
1   2    3000  劲仔食品  12.06     16.00    3125000.0    6250000.0   
2   3  600668  尖峰集团  11.68     12.42    1610300.0    3220600.0   
3   4    2615   哈尔斯   7.76     10.85    7

### 6.7 限售解禁数据

In [15]:
display_stock_fetcher_data('restricted_release', '限售解禁数据 - 限售股解禁时间表', stock)


📊 限售解禁数据 - 限售股解禁时间表
🔗 访问名称: restricted_release
📍 股票代码: 600519
📈 数据形状: 4行 x 13列
📋 列名: ['序号', '解禁时间', '解禁股东数', '解禁数量', '实际解禁数量', '未解禁数量', '实际解禁数量市值', '占总市值比例', '占流通市值比例', '解禁前一交易日收盘价', '限售股类型', '解禁前20日涨跌幅', '解禁后20日涨跌幅']

🔢 数据类型:
   序号: int64
   解禁时间: object
   解禁股东数: int64
   解禁数量: float64
   实际解禁数量: float64
   未解禁数量: int64
   实际解禁数量市值: float64
   占总市值比例: float64
   占流通市值比例: float64
   解禁前一交易日收盘价: float64
   限售股类型: object
   解禁前20日涨跌幅: float64
   解禁后20日涨跌幅: float64

📄 数据预览 (前5行):
   序号        解禁时间  解禁股东数          解禁数量        实际解禁数量  未解禁数量      实际解禁数量市值  \
0   1  2020-09-04      2  1.248316e+09  1.248316e+09      0  1.257055e+10   
1   2  2017-03-20     11  1.099461e+09  1.099461e+09      0  1.774531e+10   
2   3  2015-10-14      1  3.730694e+09  3.730694e+09      0  5.931804e+10   
3   4  2010-09-29     38  1.175407e+09  1.175407e+09      0  1.468083e+10   

     占总市值比例   占流通市值比例  解禁前一交易日收盘价       限售股类型  解禁前20日涨跌幅  解禁后20日涨跌幅  
0  0.042529  0.042529       10.07  定向增发机构配售股份  -3.893637  -6

### 6.8 大宗交易数据

In [16]:
display_stock_fetcher_data('block_trade', '大宗交易数据 - 大宗交易记录和统计', stock)


📊 大宗交易数据 - 大宗交易记录和统计
🔗 访问名称: block_trade
📍 股票代码: 600519
❌ 数据为空


## 7. 多名称访问演示

In [17]:
print("🔄 多名称访问模式演示")
print("同一个fetcher可以通过业务名称和akshare方法名访问")
print("="*60)

# 展示多名称访问的例子
multi_name_examples = [
    ('kline', 'stock_zh_a_hist', '历史行情'),
    ('info', 'stock_individual_info_em', '股票信息'),
    ('realtime', 'stock_zh_a_spot_em', '实时行情'),
    ('dividend', 'stock_dividend_detail', '分红数据'),
    ('earnings', 'stock_yjbb_em', '业绩数据')
]

for business_name, akshare_name, description in multi_name_examples:
    print(f"\n📋 {description}:")
    
    # 验证两个名称是否指向同一个fetcher实例
    if business_name in stock.fetchers and akshare_name in stock.fetchers:
        same_instance = stock.fetchers[business_name] is stock.fetchers[akshare_name]
        print(f"   业务名称: {business_name}")
        print(f"   akshare名称: {akshare_name}")
        print(f"   同一实例: {'✅ 是' if same_instance else '❌ 否'}")
        
        # 展示数据形状
        try:
            data = stock.get_data(business_name)
            print(f"   数据形状: {data.shape if not data.empty else '空数据'}")
        except:
            print(f"   数据形状: 获取失败")
    else:
        available_names = [name for name in [business_name, akshare_name] if name in stock.fetchers]
        missing_names = [name for name in [business_name, akshare_name] if name not in stock.fetchers]
        print(f"   ✅ 可用名称: {available_names}")
        print(f"   ❌ 缺失名称: {missing_names}")

print(f"\n📊 总计访问名称: {len(stock.fetchers)}个")
print(f"📊 唯一fetchers: {len(unique_fetchers)}个")

🔄 多名称访问模式演示
同一个fetcher可以通过业务名称和akshare方法名访问

📋 历史行情:
   业务名称: kline
   akshare名称: stock_zh_a_hist
   同一实例: ✅ 是
   数据形状: (242, 14)

📋 股票信息:
   业务名称: info
   akshare名称: stock_individual_info_em
   同一实例: ✅ 是


   数据形状: (8, 2)

📋 实时行情:
   ✅ 可用名称: []
   ❌ 缺失名称: ['realtime', 'stock_zh_a_spot_em']

📋 分红数据:
   ✅ 可用名称: ['dividend']
   ❌ 缺失名称: ['stock_dividend_detail']

📋 业绩数据:
   业务名称: earnings
   akshare名称: stock_yjbb_em
   同一实例: ✅ 是
   数据形状: 空数据

📊 总计访问名称: 27个
📊 唯一fetchers: 12个


## 8. 不同股票的数据对比

In [18]:
print("📊 不同股票数据对比")
print("="*60)

# 创建不同股票的实例
stocks_to_compare = [
    ('600519', '贵州茅台'),
    ('000858', '五粮液'),
    ('000001', '平安银行')
]

stock_comparison = {}

for stock_code, stock_name in stocks_to_compare:
    print(f"\n🔍 {stock_name} ({stock_code}):")
    
    try:
        # 创建股票实例
        stock_instance = StockData(stock_code)
        print(f"   实例Key: {stock_instance.key()}")
        
        # 获取基本信息
        try:
            info = stock_instance.get_data('info')
            if not info.empty:
                print(f"   基本信息: {info.shape}")
            else:
                print(f"   基本信息: 无数据")
        except:
            print(f"   基本信息: 获取失败")
        
        # 获取历史行情数据
        try:
            hist = stock_instance.get_data('kline')
            if not hist.empty:
                print(f"   历史数据: {hist.shape[0]}天 x {hist.shape[1]}列")
                stock_comparison[stock_name] = {
                    'code': stock_code,
                    'hist_days': hist.shape[0],
                    'data': hist
                }
            else:
                print(f"   历史数据: 无数据")
        except:
            print(f"   历史数据: 获取失败")
            
    except Exception as e:
        print(f"   ❌ 错误: {str(e)[:50]}...")

# 对比总结
if stock_comparison:
    print(f"\n📈 股票对比总结:")
    print("-" * 40)
    for name, info in stock_comparison.items():
        print(f"   {name} ({info['code']}): {info['hist_days']}天历史数据")
    
    # 计算平均历史数据天数
    avg_days = sum(info['hist_days'] for info in stock_comparison.values()) / len(stock_comparison)
    print(f"\n📊 平均历史数据天数: {avg_days:.0f}天")

📊 不同股票数据对比

🔍 贵州茅台 (600519):
   实例Key: 600519
   基本信息: (8, 2)
   历史数据: 242天 x 14列

🔍 五粮液 (000858):
   实例Key: 000858
Fetching stock info data!


Error fetching data: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
   基本信息: 无数据
Fetching stock history data! 000858
   历史数据: 获取失败

🔍 平安银行 (000001):
   实例Key: 000001
   基本信息: (8, 2)
   历史数据: 242天 x 14列

📈 股票对比总结:
----------------------------------------
   贵州茅台 (600519): 242天历史数据
   平安银行 (000001): 242天历史数据

📊 平均历史数据天数: 242天


## 9. 完整总览

In [19]:
print("📊 StockData完整功能总览")
print("="*80)

print(f"\n🏗️ 系统信息:")
print(f"   股票代码: {stock.symbol}")
print(f"   实例Key: {stock.key()}")

print(f"\n📈 数据获取能力:")
working_count = 0
total_data_rows = 0
basic_count = 0 
fundamental_count = 0

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    primary_name = info['names'][0]
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    
    # 分类统计
    if any(keyword in class_name.lower() for keyword in ['hist', 'info', 'chip', 'realtime']):
        category = "🔵 基础"
        basic_count += 1
    else:
        category = "🟢 基本面"
        fundamental_count += 1
    
    try:
        data = stock.get_data(primary_name)
        if not data.empty:
            working_count += 1
            total_data_rows += len(data)
            status = f"✅ {data.shape[0]}行 x {data.shape[1]}列"
        else:
            status = "⚠️ 空数据"
    except Exception as e:
        status = f"❌ 错误"
    
    print(f"   {i:2d}. {category} {class_name}: {status}")
    print(f"       访问名称: {all_names}")

print(f"\n🎯 统计结果:")
print(f"   正常工作的fetchers: {working_count}/{len(unique_fetchers)}")
print(f"   基础数据fetchers: {basic_count}个")
print(f"   基本面数据fetchers: {fundamental_count}个")
print(f"   总数据行数: {total_data_rows:,}行")
print(f"   访问名称总数: {len(stock.fetchers)}个")
print(f"   成功率: {working_count/len(unique_fetchers)*100:.1f}%")

print(f"\n💡 使用建议:")
print(f"   • 使用业务名称访问更直观: stock.kline, stock.dividend")
print(f"   • 使用akshare名称保持兼容性: stock.stock_zh_a_hist")
print(f"   • 通过get_data()方法统一访问: stock.get_data('kline')")
print(f"   • 创建不同股票实例获取对应数据: StockData('600519')")
print(f"   • 所有数据都支持pandas DataFrame操作")

print(f"\n🔍 主要应用场景:")
print(f"   • 技术分析 - 基于历史行情和筹码分布进行技术分析")
print(f"   • 基本面分析 - 通过财务数据和业绩指标评估投资价值")
print(f"   • 投资决策 - 结合分红、股东等信息制定投资策略")
print(f"   • 风险评估 - 通过限售解禁、大宗交易等数据评估风险")
print(f"   • 实时监控 - 使用实时行情数据进行盘中监控")

📊 StockData完整功能总览

🏗️ 系统信息:
   股票代码: 600519
   实例Key: 600519

📈 数据获取能力:


    1. 🔵 基础 StockHistFetcher: ✅ 242行 x 14列
       访问名称: kline | stock_zh_a_hist
    2. 🔵 基础 StockInfoFetcher: ✅ 8行 x 2列
       访问名称: info | stock_individual_info_em
    3. 🔵 基础 StockChipFetcher: ✅ 90行 x 9列
       访问名称: chip | stock_cyq_em
Fetching stock realtime data!
Error fetching data: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
    4. 🔵 基础 StockRealtimeFetcher: ⚠️ 空数据
       访问名称: bid_ask | stock_bid_ask_em
    5. 🟢 基本面 BlockTradeFetcher: ⚠️ 空数据
       访问名称: block_trade | stock_block_trade_big_em
    6. 🟢 基本面 DividendFetcher: ⚠️ 空数据
       访问名称: dividend | stock_fhps_em
    7. 🟢 基本面 EarningsFetcher: ⚠️ 空数据
       访问名称: earnings | stock_yjbb_em | stock_yjyg_em
    8. 🟢 基本面 FinancialStatementsFetcher: ⚠️ 空数据
       访问名称: financial_statements | stock_lrb_em | stock_zcfz_em | stock_xjll_em


    9. 🟢 基本面 RepurchaseFetcher: ✅ 4868行 x 18列
       访问名称: repurchase | stock_repurchase_em
   10. 🟢 基本面 RestrictedReleaseFetcher: ✅ 4行 x 13列
       访问名称: restricted_release | stock_restricted_release_queue_em
   11. 🟢 基本面 ShareChangeFetcher: ⚠️ 空数据
       访问名称: share_change | stock_share_change_em


   12. 🟢 基本面 TopShareholdersFetcher: ⚠️ 空数据
       访问名称: top_shareholders | stock_gdfx_free_top_10_em

🎯 统计结果:
   正常工作的fetchers: 5/12
   基础数据fetchers: 4个
   基本面数据fetchers: 8个
   总数据行数: 5,212行
   访问名称总数: 27个
   成功率: 41.7%

💡 使用建议:
   • 使用业务名称访问更直观: stock.kline, stock.dividend
   • 使用akshare名称保持兼容性: stock.stock_zh_a_hist
   • 通过get_data()方法统一访问: stock.get_data('kline')
   • 创建不同股票实例获取对应数据: StockData('600519')
   • 所有数据都支持pandas DataFrame操作

🔍 主要应用场景:
   • 技术分析 - 基于历史行情和筹码分布进行技术分析
   • 基本面分析 - 通过财务数据和业绩指标评估投资价值
   • 投资决策 - 结合分红、股东等信息制定投资策略
   • 风险评估 - 通过限售解禁、大宗交易等数据评估风险
   • 实时监控 - 使用实时行情数据进行盘中监控
